In [ ]:
!pip install kaggle

In [ ]:
!pip3 install wandb

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

100% 1.31G/1.32G [00:24<00:00, 95.7MB/s]
100% 1.32G/1.32G [00:24<00:00, 56.7MB/s]


In [6]:
!unzip -qq gender-recognition-200k-images-celeba.zip

In [8]:
import wandb
import numpy as np
import cv2
import tensorflow as tf
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
wandb.init(project='gender-recognition')

In [10]:
batch_size = 256
lr = 0.001
epoch = 40
width = height = 64

In [11]:
data_generator = ImageDataGenerator(
                                  rescale = 1 / 255,
                                  horizontal_flip = True
                                  )

train_data = data_generator.flow_from_directory("Dataset/Train/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

val_data = data_generator.flow_from_directory("Dataset/Validation/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

test_data = data_generator.flow_from_directory("Dataset/Test/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([

    Conv2D(96, (11, 11), activation= 'relu', input_shape = (width, height, 3)),
    BatchNormalization(),
    MaxPooling2D(2),


    Conv2D(256, (11, 11), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(256, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),
    MaxPooling2D(2, strides= (2, 2)),


    Flatten(),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(1, activation='sigmoid')                          
])

In [12]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 2s 0us/step


In [13]:
for layer in base_model.layers[:-8]:
  layer.trainable = False

In [14]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(2, activation='softmax')                          
])

In [15]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics='accuracy')

In [16]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 10,
          callbacks = [WandbCallback()]
)

Epoch 1/10
625/625 [==============================] - 258s 353ms/step - loss: 0.3551 - accuracy: 0.8458 - val_loss: 0.3035 - val_accuracy: 0.8684


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
625/625 [==============================] - 218s 349ms/step - loss: 0.2841 - accuracy: 0.8783 - val_loss: 0.3137 - val_accuracy: 0.8686
Epoch 3/10
625/625 [==============================] - 217s 347ms/step - loss: 0.2628 - accuracy: 0.8880 - val_loss: 0.2997 - val_accuracy: 0.8707
Epoch 4/10
625/625 [==============================] - 217s 347ms/step - loss: 0.2457 - accuracy: 0.8973 - val_loss: 0.2861 - val_accuracy: 0.8790
Epoch 5/10
625/625 [==============================] - 215s 344ms/step - loss: 0.2302 - accuracy: 0.9044 - val_loss: 0.2788 - val_accuracy: 0.8827
Epoch 6/10
625/625 [==============================] - 215s 343ms/step - loss: 0.2153 - accuracy: 0.9115 - val_loss: 0.2884 - val_accuracy: 0.8802
Epoch 7/10
625/625 [==============================] - 214s 342ms/step - loss: 0.2027 - accuracy: 0.9174 - val_loss: 0.2871 - val_accuracy: 0.8831
Epoch 8/10
625/625 [==============================] - 213s 341ms/step - loss: 0.1855 - accuracy: 0.9254 - val_loss: 0.3055 -

In [ ]:
model.save('/content/drive/MyDrive/gender_model_resnet.h5')

In [18]:
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis= 1)

print(confusion_matrix(test_data.classes, y_pred))
targets = set(test_data.class_indices.keys())
print('classification_report:')
print(classification_report(test_data.classes, y_pred, target_names= targets))

[[6374 5168]
 [4716 3743]]
classification_report:
              precision    recall  f1-score   support

      Female       0.57      0.55      0.56     11542
        Male       0.42      0.44      0.43      8459

    accuracy                           0.51     20001
   macro avg       0.50      0.50      0.50     20001
weighted avg       0.51      0.51      0.51     20001

